# API Request for Connection to Sentinel Hub <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:**
[S1GRD](http://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-1/Data_products)

*Dataset is external to the Digital Earth Africa platform.*


## Introduction to Sentinel Hub

Sentinel Hub is a multi-spectral and multi-temporal big data satellite imagery service, capable of fully automated archiving, real-time processing and distribution of remote sensing data and related Earth Observation products.It is an engine for processing of petabytes of satellite data that opens the door for machine learning and helping hundreds of application developers worldwide. It makes Sentinel, Landsat, and other Earth observation imagery easily accessible for browsing, visualization and analysis. Sentinel Hub is operated by Sinergise company that have developed enterprise level spatial solutions for governmental clients in Europe and Africa. Sinergise is headquartered in Ljubljana, Slovenia, and has subsidiaries in Czech Republic, Moldova and Serbia. More information on Sentinel Hub can be found [here](https://www.sentinel-hub.com/explore).
Users can use available APIs to retrieve satellite data over their Area of Interest (AOI) and specific time range from full archives in a matter of seconds.

## Sentinel-1AB Collection 

The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) imagery is provided by two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging, enabling them to acquire imagery regardless of the weather. Main applications are for monitoring sea ice, oil spills, marine winds, waves & currents, land-use change, land deformation among others, and to respond to emergencies such as floods and earthquakes. Sentinel Hub currently supports Sentinel-1 Level-1 GRD (Ground Range Detected) products only.

## Access to Sentinel Hub Data and Services

The Sentinel Hub API is a RESTful API interface to various satellite [imagery archives](https://docs.sentinel-hub.com/api/latest/#/data/). It provides access to raw satellite data, rendered images, statistical analysis and much more.In order to access and use sentinel hub services through API calls, the user needs to take the following steps:

- Register to  [Seninel Hub](https://www.sentinel-hub.com/) portal 
- [Authentication](https://docs.sentinel-hub.com/api/latest/#/API/authentication)
- [Process](https://docs.sentinel-hub.com/api/latest/#/Evalscript/) 

Following registration to the Sentinel Hub, the user will be allocated a non-commercial account with dynamic dashboard that shows the subscription plan, processing units and the number of requests per minute.

To access data user needs to send a request to process API. The requested data will be returned as the response to this request. Each request can be tailored to get the user exactly the data that was requested. To do this requires setting various parameters which depend on the data source the user is querying. The following examples help the user to understand the parameters for S1GRD data. To explore more examples of such requests go [here](https://docs.sentinel-hub.com/api/latest/#/data/Examples_for_S1GRD), and for an overview of all API parameters see the [API Reference](https://docs.sentinel-hub.com/api/latest/reference/).

## Data Availibility 

Sentinel Hub API is available in several deployments including:
- Main Sentinel Hub deployment - EU-1-Central region 
- Main Sentinel Hub deployment - US-West-2 region
- CreoDIAS Sentinel Hub deployment
- EOCloud Sentinel Hub deployment

The hub can be searched and accessed throught the API [endpoints](https://docs.sentinel-hub.com/api/latest/#/data/). 
Synergise has two major pipelines for Sentinel-1 GRD data: (a) EU-1-Central region that covers global archive since 2017 and (b) EOCloud that covers the global archive since 2014. Both pipelines are available through [Sentinel Hub API](https://docs.sentinel-hub.com/api/latest/#/) that supports processing a typical 512x512 px tile in one second aka “live” processing and [batch processing](https://docs.sentinel-hub.com/api/latest/#/BATCH_API/batch_processor) using the latest technology.

## Available Bands and Data

The following information in important when defining input object in evalscript. Any string listed in the column Name can be an element of the input.bands array in your evalscript.

### Import Dependencies

In [1]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [29]:
client_id = ''
client_secret = ''

In [30]:
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

In [31]:
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token', client_id=client_id,client_secret=client_secret)

In [32]:
resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

### Two-month Temporal Average on S1GRD Orthorectified Gamma0 in VV polarization

In [33]:
url = "https://services.sentinel-hub.com/api/v1/process" 
headers = {
    "Authorization": f"Bearer {token['access_token']}"
}
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["VV", "dataMask"],
    output: {id:"default", bands: 1},
    mosaicking: Mosaicking.ORBIT
    }
}

function evaluatePixel(samples) {
  return [calculateAverage(samples)]
}

function calculateAverage(samples) {
  var sum = 0;
  var nValid = 0;
  for (let i = 0; i < samples.length; i++) {
    var sample = samples[i];
    if (sample.dataMask != 0) {
      nValid++;
      sum += toDb(sample.VV);
    }
  }
  return sum / nValid;
}

function toDb(linear) {
  return Math.max(0, Math.log(linear) * 0.21714724095 + 1)
}
""".rstrip()

data = {
    "input": {
        "bounds":{
            "bbox":[
                1360000,5121900,1370000,5131900
            ],
            "properties":{
                "crs":"http://www.opengis.net/def/crs/EPSG/0/3857"
            }
        },
        "data": [
            {
                "type":"S1GRD",
                "dataFilter":{
                    "timeRange":{
                        "from":"2020-02-01T00:00:00Z",
                        "to":"2020-04-02T23:59:59Z"
                    },
                    "orbitDirection": "ASCENDING"
                },
                "processing":{
                    "orthorectify": "true"
                }
            }
        ]
    },
    "output": {
        "width":512,
        "height":512,
        "response":[
            {
                "identifier": "default",
                "format":{
                    "type":"image/png"
                }
            }
        ]
    },
    "evalscript": evalscript
}
response = requests.post(url, json=data, headers=headers)
print(response)

<Response [200]>


In [34]:
if response.status_code == 200:
    with open('TemporalAverage_S1_VV2.tif', 'wb') as f:
        for chunk in response:
            f.write(chunk)